In [1]:
%load_ext autoreload
%autoreload 2

# 1. Rendering templates from train_pbr

In [5]:
import hydra
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

# Initialize Hydra and compose the configuration
initialize(config_path="configs")
cfg = compose(config_name="run_inference")
OmegaConf.set_struct(cfg, False)


/tmp/ipykernel_1388265/392793680.py:6: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="configs")


In [9]:
import os
import glob

dataset = "daoliuzhao"

default_ref_dataloader_config = cfg.data.reference_dataloader
ref_dataloader_config = default_ref_dataloader_config.copy()
ref_dataloader_config._target_ = "src.dataloader.bop_pbr.BOPTemplatePBR"
ref_dataloader_config.root_dir = f"datasets/bop23_challenge/datasets/{dataset}"
ref_dataloader_config.template_dir = f"datasets/bop23_challenge/datasets/templates_pyrender/{dataset}"
ref_dataloader_config.level_templates = 0 # Level 0 :42 templates, level 1, 162 templates
ref_dataloader_config.split = "train_pbr"

ply_files = glob.glob(f"datasets/bop23_challenge/datasets/{dataset}/models/*.ply")
obj_ids = sorted([int(os.path.basename(obj_id)[8:10]) for obj_id in ply_files])
ref_dataloader_config.obj_ids = obj_ids

out_folder = "xoa3"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
ref_dataloader_config.out_dir = out_folder
ref_dataloader_config.templates_output_folder = f"{out_folder}/real_images_templates"


In [10]:
from hydra.utils import instantiate
ref_dataset = instantiate(ref_dataloader_config) # src.dataloader.bop.BOPTemplatePBR
ref_dataset.load_processed_metaData(reset_metaData=True)

Loading metaData: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]
Finding nearest rendering close to template poses: 100%|██████████| 1/1 [00:00<00:00, 159.03it/s]


In [16]:
import pandas as pd
templates = pd.read_csv(f"datasets/bop23_challenge/datasets/daoliuzhao/train_pbr_metaData.csv")

In [13]:
len(templates)

4875

In [17]:
from torchvision.utils import save_image
import pandas as pd

# templates = pd.read_csv(f"{out_folder}/train_pbr_processed_metaData.csv")

# Save templates as images
for ob_id in range(1,len(obj_ids)+1): # Change to  42, 162, 462 for level templates 0, 1, 2
    obj_id_path = os.path.join(f"{out_folder}/synthetic_images_templates/{dataset}/train_pbr", f'obj_{(ob_id):06d}') # The transformed templates - check line 244, 245 in bop_pbr.py
    original_obj_id_path = os.path.join(f"{out_folder}/synthetic_images_templates/{dataset}/train_pbr", f'obj_{(ob_id):06d}_original')
    
    if not os.path.exists(obj_id_path):
        os.makedirs(obj_id_path)
    if not os.path.exists(original_obj_id_path):
        os.makedirs(original_obj_id_path)

    for i, img in enumerate(ref_dataset[ob_id-1]["templates"]): # all the crop her is normalize and resize to 224*224 alsready in __getitem__
        save_image(img, os.path.join(obj_id_path, f'{i:06d}.png'))
    for i, img in enumerate(ref_dataset[ob_id-1]["unnormalized_templates"]): # all the crop her is normalize and resize to 224*224 alsready in __getitem__
        save_image(img, os.path.join(original_obj_id_path, f'{i:06d}.png'))

In [ ]:
from PIL import Image

def combine_images(image_paths, output_path, image_size=(224, 224), grid_size=(6, 7)):
    rows, cols = grid_size
    if len(image_paths) != rows * cols:
        raise ValueError("Incorrect number of images.")
    images = [Image.open(path).resize(image_size) for path in image_paths]
    total_width = cols * image_size[0]
    total_height = rows * image_size[1]
    combined_image = Image.new("RGB", (total_width, total_height))
    for index, image in enumerate(images):
        x = (index % cols) * image_size[0]
        y = (index // cols) * image_size[1]
        combined_image.paste(image, (x, y))
    combined_image.save(output_path)

image_paths = glob.glob("xoa3/synthetic_images_templates/daoliuzhao/train_pbr/obj_000001_original/*.png")
#datasets/bop23_challenge/datasets/templates_pyrender/daoliuzhao/obj_000001)
output_path = "xoa3/synthetic_images_templates/daoliuzhao/42_pbr_templates.png"
combine_images(image_paths, output_path)

In [28]:
from PIL import Image

def combine_images_pyrender(image_paths, output_path, image_size=(100, 100), grid_size=(6, 7)):
    rows, cols = grid_size
    if len(image_paths) != rows * cols:
        raise ValueError("Incorrect number of images.")
    images = []
    for path in image_paths:
        img = Image.open(path)
        width, height = img.size
        crop_width, crop_height = int(width * 0.4), int(height * 0.4)
        left = (width - crop_width) // 2
        top = (height - crop_height) // 2
        right = left + crop_width
        bottom = top + crop_height
        cropped = img.crop((left, top, right, bottom)).resize(image_size)
        images.append(cropped)
    total_width = cols * image_size[0]
    total_height = rows * image_size[1]
    combined_image = Image.new("RGB", (total_width, total_height))
    for index, image in enumerate(images):
        x = (index % cols) * image_size[0]
        y = (index // cols) * image_size[1]
        combined_image.paste(image, (x, y))
    combined_image.save(output_path)

image_paths = glob.glob("datasets/bop23_challenge/datasets/templates_pyrender/daoliuzhao/obj_000001/*.png")
output_path = "xoa3/synthetic_images_templates/daoliuzhao/42_pyrender_templates.png"
combine_images_pyrender(image_paths, output_path)
